# RAG system with Feedback loop: Enhancing Retrieval and Response Quality
## 1. Overview
- This system implements a Retrieval-Augmented Generation (RAG) approach with an **integrated feedback loop**.
- It aims to improve the quality and relevance of responses over time by incorporating user feedback and dynamically adjusting the retrieval process.

## 2. Motivation
- Traditional RAG systems can sometimes produce inconsistent or irrelevant responses due to limitations in the retrieval process or the underlying knowledge base. By implementing a feedback loop, we can:
  - Continuously improve the quality of retrieved documents
  - Enhance the relevance of generated responses
  - Adapt the system to user preferences and needs over time


##### Import relevant libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import os
import sys
import json

from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks

from helper_functions import *
from evaluation.evalute_rag import *

from IPython.display import display, Markdown, Latex

##### Define the document path and environment variables

In [3]:
path = "../data/vks/pdf/vi/01-vks-la-gi.pdf"

ENV_FILE_PATH = "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
MAX_TOKENS = 32700

envs = load_env_to_dict(ENV_FILE_PATH)

##### Create vector store and retrieval QA chain


In [4]:
content = read_pdf_to_string(path)
vectorstore = encode_from_string(content)
retriever = vectorstore.as_retriever()

llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=envs['VLLM_HOST_URL_2'] + "/v1/",
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    streaming=True,
    stop=["<|eot_id|>",'<|eom_id|>'],
)

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Function to format user feedback in a dictionary

In [5]:
def get_user_feedback(query, response, relevance, quality, comments=""):
    return {
        "query": query,
        "response": response,
        "relevance": int(relevance),
        "quality": int(quality),
        "comments": comments
    }

##### Function to store the feedback in a json file


In [6]:
def store_feedback(feedback):
    with open("../data/feedback_data.json", "a") as f:
        json.dump(feedback, f)
        f.write("\n")

##### Function to read the feedback file


In [7]:
def load_feedback_data():
    feedback_data = []
    try:
        with open("../data/feedback_data.json", "r") as f:
            for line in f:
                feedback_data.append(json.loads(line.strip()))
    except FileNotFoundError:
        print("No feedback data file found. Starting with empty feedback.")
    return feedback_data

##### Function to adjust files relevancy based on the feedbacks file


In [8]:
class Response(BaseModel):
    answer: str = Field(..., title="The answer to the question. The options can be only 'Yes' or 'No'")

def adjust_relevance_scores(query: str, docs: List[Any], feedback_data: List[Dict[str, Any]]) -> List[Any]:
    # Create a prompt template for relevance checking
    relevance_prompt = PromptTemplate(
        input_variables=["query", "feedback_query", "doc_content", "feedback_response"],
        template="""
        Determine if the following feedback response is relevant to the current query and document content.
        You are also provided with the Feedback original query that was used to generate the feedback response.
        Current query: {query}
        Feedback query: {feedback_query}
        Document content: {doc_content}
        Feedback response: {feedback_response}

        Is this feedback relevant? Respond with only 'Yes' or 'No'.
        """
    )
    llm = ChatOpenAI(
        openai_api_key="EMPTY",
        openai_api_base=envs['VLLM_HOST_URL_2'] + "/v1/",
        model_name=MODEL_NAME,
        max_tokens=MAX_TOKENS,
        streaming=True,
        stop=["<|eot_id|>",'<|eom_id|>'],
    )

    # Create an LLMChain for relevance checking
    relevance_chain = relevance_prompt | llm.with_structured_output(Response)

    for doc in docs:
        relevant_feedback = []

        for feedback in feedback_data:
            # Use LLM to check relevance
            input_data = {
                "query": query,
                "feedback_query": feedback['query'],
                "doc_content": doc.page_content[:1000],
                "feedback_response": feedback['response']
            }
            result = relevance_chain.invoke(input_data).answer

            if result == 'yes':
                relevant_feedback.append(feedback)

        # Adjust the relevance score based on feedback
        if relevant_feedback:
            avg_relevance = sum(f['relevance'] for f in relevant_feedback) / len(relevant_feedback)
            doc.metadata['relevance_score'] *= (avg_relevance / 3)  # Assuming a 1-5 scale, 3 is neutral

    # Re-rank documents based on adjusted scores
    return sorted(docs, key=lambda x: x.metadata['relevance_score'], reverse=True)

##### Function to fine tune the vector index to include also queries + answers that received good feedbacks


In [9]:
def fine_tune_index(feedback_data: List[Dict[str, Any]], texts: List[str]) -> Any:
    # Filter high-quality responses
    good_responses = [f for f in feedback_data if f['relevance'] >= 4 and f['quality'] >= 4]

    # Extract queries and responses, and create new documents
    additional_texts = []
    for f in good_responses:
        combined_text = f['query'] + " " + f['response']
        additional_texts.append(combined_text)

    # make the list a string
    additional_texts = " ".join(additional_texts)

    # Create a new index with original and high-quality texts
    all_texts = texts + additional_texts
    new_vectorstore = encode_from_string(all_texts)

    return new_vectorstore

##### Demonstration of how to retrieve answers with respect to user feedbacks


In [10]:
query = "VKS là gì?"

# Get response from RAG system
response = qa_chain.invoke(query)["result"]

relevance = 5
quality = 5

# Collect feedback
feedback = get_user_feedback(query, response, relevance, quality)

# Store feedback
store_feedback(feedback)

# Adjust relevance scores for future retrievals
docs = retriever.invoke(query)
adjusted_docs = adjust_relevance_scores(query, docs, load_feedback_data())

# Update the retriever with adjusted docs
retriever.search_kwargs['k'] = len(adjusted_docs)
retriever.search_kwargs['docs'] = adjusted_docs

In [14]:
display(Markdown(response))

Được rồi, người dùng đang hỏi "VKS là gì?". Từ thông tin mà tôi có, VKS là dịch vụ Kubernetes được quản lý bởi VNGCloud. Tôi cần giải thích rõ ràng và chi tiết để người dùng hiểu được VKS và những điểm nổi bật của nó.

Đầu tiên, tôi应该 định nghĩa VKS là gì, giải thích rằng nó là dịch vụ Kubernetes được quản lý, giúp triển khai và quản lý ứng dụng container dễ dàng. Sau đó, tôi nên trình bày các điểm nổi bật như quản lý Control Plane tự động, hỗ trợ phiên bản mới, tích hợp Calico CNI, tự động nâng cấp và mở rộng, tiết kiệm chi phí, tích hợp storage và load balancer, cũng như bồi mật.

Ngoài ra, kể về các region mà VKS cung cấp ở Hà Nội và Hồ Chí Minh也是 một điểm quan trọng. Cuối cùng, tôi cần đề cập đến các ưu điểm như dễ sử dụng và chi phí hợp lý, đồng thời cung cấp đường link để tham khảo thêm. Tôi phải sắp xếp thông tin này một cách logic và dễ hiểu để người dùng nắm rõ được VKS là gì và lợi ích của nó.
</think>

VNGCloud Kubernetes Service (VKS) là một dịch vụ quản lý Kubernetes được cung cấp bởi VNGCloud, giúp triển khai và quản lý các ứng dụng container một cách dễ dàng và hiệu quả. Dưới đây là một số điểm nổi bật của VKS:

1. **Quản lý Control Plane tự động**: VKS miễn phí quản lý Control Plane, giúp người dùng tập trung vào phát triển ứng dụng.
2. **Hỗ trợ phiên bản mới**: Cung cấp các phiên bản Kubernetes mới nhất (từ 1.27, 1.28, 1.29).
3. **Tích hợp Calico CNI**: Mang lại hiệu suất và bảo mật cao trong Kubernetes Networking.
4. **Nâng cấp dễ dàng**: Hỗ trợ nâng cấp phiên bản Kubernetes một cách nhanh chóng và không gây gián đoạn.
5. **Tự động mở rộng và sửa lỗi**: Tự động mở rộng Node Group và tự động sửa lỗi khi có vấn đề về node.
6. **Tiết kiệm chi phí và độ tin cậy cao**: Triển khai Control Plane với độ sẵn sàng cao và miễn phí.
7. **Tích hợp Blockstore Native**: Quản lý Blockstore thông qua Kubernetes CSI, hỗ trợ các tính năng như thay đổi kích thước, IOPS và snapshot.
8. **Tích hợp Load Balancer**: Quản lý NLB/ALB dễ dàng thông qua Kubernetes sử dụng các driver tích hợp sẵn.
9. **Bảo mật cao**: Hỗ trợ tạo Private Node Group với kiểm soát truy cập dựa trên Whitelist IP.

VKS hiện tại được cung cấp tại hai cơ sở hạ tầng ở Hà Nội và Hồ Chí Minh, cho phép người dùng lựa chọnbased on nhu cầu. Để tìm hiểu thêm, có thể tham khảo tại [đây](https://docs.vngcloud.vn/vng-cloud- document/vn/vks/vks-la-gi).

In [15]:
for i, doc in enumerate(adjusted_docs):
    print(f"Document {i+1}: {doc.id} - {doc.metadata}")
    display(Markdown(doc.page_content))

Document 1: 4c1f41cf-ca98-478b-b099-f76459539784 - {'relevance_score': 1.0}


1. Giới thiệu về VNGCloud Kubernetes
Service - VKS
1.1. VNGCloud Kubernetes Service (VKS) là gì?
VNGCloud Kubernetes Service (hay còn gọi là VKS) là một dịch vụ quản lý Kubernetes được cung
cấp bởi VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách
dễ dàng và hiệu quả. VKS giúp bạn tập trung vào việc phát triển ứng dụng mà không cần quan tâm
đến việc quản lý Control Plane của Kubernetes.
VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn
giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền
tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các
ứng dụng container trên môi trường phân tán.
1.2. Những điểm nổi bật của VKS
Những điểm nổi bật của dịch vụ VKS có thể kể đến gồm:
Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng
bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát
triển ứng dụng.
Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản Kubernetes
mới nhất (minor version từ 1.27, 1.28, 1.29) để đảm bảo bạn luôn tận dụng được những tính
năng tiên tiến nhất.
Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật cao.
Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ dàng
và nhanh chóng, giúp bạn luôn cập nhật những cải tiến mới nhất.
Scaling & Healing Automatically: VKS tự động mở rộng Node group khi cần thiết và tự động
sửa lỗi khi node gặp vấn đề, giúp bạn tiết kiệm thời gian và công sức quản lý.
Giảm chi phí và nâng cao độ tin cậy: VKS triển khai Control Plane của Kubernetes ở chế độ
sẵn sàng cao và hoàn toàn miễn phí, giúp bạn tiết kiệm chi phí và nâng cao độ tin cậy cho hệ
thống.
Tích hợp Blockstore Native (Container Storage Interface - CSI): VKS cho phép bạn quản lý
Blockstore thông qua YAML của Kubernetes, cung cấp lưu trữ bền vững cho container và hỗ
trợ các tính năng quan trọng như thay đổi kích thước, thay đổi IOPS và snapshot volume.
Tích hợp Load Balancer (Network Load Balancer, Application Load Balancer) thông qua
các driver được tích hợp sẵn như VNGCloud Controller Mananger, VNGCloud Ingress
Controller: VKS cung cấp khả năng quản lý NLB/ALB thông qua YAML của Kubernetes, giúp
bạn dễ dàng expose Service trong Kubernetes ra bên ngoài.
Nâng cao bảo mật: VKS cho phép bạn tạo Private Node Group với chỉ Private IP và kiểm soát
quyền truy cập vào cluster thông qua tính năng Whitelist IP, đảm bảo an toàn cho hệ thống của
bạn.
Ngoài ra, VKS còn có các ưu điểm sau:
Dễ sử dụng: VKS cung cấp giao diện đơn giản và dễ sử dụng.
Chi phí hợp lý: VKS cung cấp mức giá cạnh tranh cho các dịch vụ của mình.
1.3. VKS cung cấp dịch vụ trên các khu vực (region)
nào?
Hiện tại, trên VKS, chúng tôi đang cung cấp cho bạn 2 cơ sở hạ tầng riêng biệt được đặt tại Hà Nội
và Hồ Chí Minh. Bạn có thể lựa chọn sử dụng VKS trên mỗi region tùy thuộc vào vị trí và nhu cầu
thực tế của bạn. Đối với 2 farm HCM03, HAN01, các thông số cụ thể cho mỗi region được chúng
tôi cung cấp như sau:
Khu vực Hồ Chí Minh (HCM03): https://vks.console.vngcloud.vn
Khu vực Hà Nội (HAN01): https://vks-han-1.console.vngcloud.vn
1.4. Tài liệu tham khảo:
Bạn có thể tìm hiểu thêm về VKS tại đường dẫn https://docs.vngcloud.vn/vng-cloud-
document/vn/vks/vks-la-gi

##### Finetune the vectorstore periodicly


In [16]:
# Periodically (e.g., daily or weekly), fine-tune the index
new_vectorstore = fine_tune_index(load_feedback_data(), content)
retriever = new_vectorstore.as_retriever()